# Basic topographic features

*This lesson made by Simon M Mudd and last updated 30/09/2021*

In this lesson we are going to download some data and look at basic features of channels and river basins. 

*LESSON SHORTCUT*

If you want to run a full analysis of a place of your choosing just modify the (tbc)

## First import some stuff we need

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import pandas as pd
import geopandas as gpd
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

## Grab some data

First, we need to grab some data. We use a tool in `lsdviztools.lsdbasemaptools` called the `ot_scraper` (`ot` is for opentopography.org).

We want to make sure the `ot_scraper` is up to date, so use the following command to update the `lsdviztools` package. 

In [ ]:
!pip install lsdviztools --upgrade

Below is some code for grabbing data from opentopography. It uses the syntax:

`DEM_name = bmt.ot_scraper(parameters=...)`

where you need to give it some paramaters. These are things like what sort of data you want (most people will use the SRTM 30 metre data) and you also tell it the lower left and the upper right corners, using latitude and longitude. You can get these from goole earth by right clicking on the map and selecting "what's here". 

In this example I use the Rio Aguas in southern Spain. 

In [ ]:
Aguas_DEM = bmt.ot_scraper(source = "SRTM30",
                           longitude_W = -2.318307057720176, longitude_E = -1.8425313329873874, 
                           latitude_S = 36.990554387425014, latitude_N = 37.23367133834253,
                           prefix = "rio_aguas") 
Aguas_DEM.print_parameters()
Aguas_DEM.download_pythonic()

We need to convert this data into a format that can be understood by *lsdtopotools*, which is a topographic analysis package developed at the University of Edinburgh. 

In [ ]:
DataDirectory = "./"
RasterFile = "rio_aguas_SRTMGL1.tif"
gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.01,resolution=30)

# Now lets do some basic topographic analysis

For this to work you need *lsdtopotools* installed on your system. But if you are using either Edinburgh's noteable service or the *lsdtopotools* docker container, it is already installed. If you are not using one of these, you will need to install it using conda (see *lsdtopotools* installation instructions. 

We will extract some topogaphic metrics using `lsdtopotools`. 

The `lsdtt_parameters` are the various parameters that you can use to run an analysis. We will discuss these later. For now, we will just follow this recipe. 

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "rio_aguas_SRTMGL1_UTM",
                                 write_prefix= "rio_aguas_SRTMGL1_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

# Plot some data

We are now going to do some simple plots using a mapping package that we put together. There are more general ways to visualise data, but this makes pretty pictures quickly.  

In [ ]:
%matplotlib inline
Base_file = "rio_aguas_SRTMGL1_UTM"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")

In [ ]:
Base_file = "rio_aguas_SRTMGL1_UTM"
Drape_prefix = "rio_aguas_SRTMGL1_UTM_SLOPE"
DataDirectory = "./"
img_name2 = lsdmw.SimpleDrape(DataDirectory,Base_file, Drape_prefix, 
                              cmap = "bwr", cbar_loc = "right", 
                              cbar_label = "Gradient (m/m)",
                              save_fig=False, size_format="geomorphology",
                              colour_min_max = [0,1.25])

# Get some channel profiles

Okay, we will now run a different analysis. We will get some channel profiles. 

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "print_chi_data_maps" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "rio_aguas_SRTMGL1_UTM",
                                 write_prefix= "rio_aguas_SRTMGL1_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

We can look to see what files we have using the following command. the `!` tells this notebook to run a command on the underlying linux operating system, and `ls` in linux is a command to list files. 

In [ ]:
!ls

The file with the channels is the one with `chi_data_map` in the filename. We are going to load this into a `pandas` dataframe. You can think of `pandas` as a kind of excel for python. It does data handling of spreadsheet-like information (and loads more.)

In [ ]:
df = pd.read_csv("rio_aguas_SRTMGL1_UTM_chi_data_map.csv")
df.head()

In [ ]:
%matplotlib inline
fname_prefix = "rio_aguas_SRTMGL1_UTM"
ChannelFileName = "rio_aguas_SRTMGL1_UTM_chi_data_map.csv"
DataDirectory = "./"
lsdmw.PrintChiChannelsAndBasins(DataDirectory,fname_prefix, ChannelFileName, 
                                add_basin_labels = True, cmap = "jet", cbar_loc = "right", 
                                size_format = "ESURF", fig_format = "png", 
                                dpi = 250,plotting_column = "elevation")

In [ ]:
from IPython.display import Image
Image('rio_aguas_SRTMGL1_UTM_chi_channels_and_basins.png')

In [ ]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf = gdf.set_crs(epsg=4326)
print(gdf.head())

gdf2 = gdf.to_crs("EPSG:32610")

gdf_b1 = gdf2[(gdf2['basin_key'] == 0)]
print(gdf_b1.head())

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.scatter(gdf_b1.chi,gdf_b1.elevation,c=gdf_b1.flow_distance,s=1)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
fig.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.scatter(gdf_b1.flow_distance,gdf_b1.elevation,c=gdf_b1.flow_distance,s=1)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
fig.show()